---
title: "NBA Shot Chart Scraper"
author: Paul Grajzl
format: html
jupyter: python3
---

In [ ]:
import os
import pandas as pd
from nba_api.stats.endpoints import shotchartdetail
from nba_api.stats.static import players
from time import sleep
from unidecode import unidecode

# --- Define LeBron's full career seasons ---
lebron_seasons = [f"{year}-{str(year + 1)[-2:]}" for year in range(2003, 2024)]

# --- Normalize and fetch player ID ---
def get_lebron_id():
    all_players = players.get_players()
    for p in all_players:
        if unidecode(p['full_name']).lower() == "lebron james":
            return p['id']
    raise ValueError("Could not find LeBron James in player database.")

# --- Create output directory ---
lebron_folder = "LeBron_James"
os.makedirs(lebron_folder, exist_ok=True)

# --- Download shot data for each season ---
lebron_id = get_lebron_id()

for season in lebron_seasons:
    try:
        shots = shotchartdetail.ShotChartDetail(
            team_id=0,  # all teams he's played for
            player_id=lebron_id,
            season_type_all_star="Regular Season",
            season_nullable=season
        )
        df = shots.get_data_frames()[0]

        if not df.empty:
            filepath = os.path.join(lebron_folder, f"{season}.csv")
            df.to_csv(filepath, index=False)
            print(f"Saved LeBron James - {season}")

        sleep(1)  # throttle to avoid API issues
    except Exception as e:
        print(f"Error fetching data for {season}: {e}")
        sleep(2)
